<a href="https://colab.research.google.com/github/mellow-75/hackathon-/blob/main/train_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.ensemble
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score ,f1_score,confusion_matrix
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras import Model

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/hackathon/Doceree-HCP_Train.csv", encoding = "latin=1")

In [ ]:
df.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


In [ ]:
print("no of nan values : " ,df['ID'].isna().sum())
print(df.shape)

no of nan values :  0
(113937, 14)


In [ ]:
print("no of nan values : " ,df['DEVICETYPE'].isna().sum())
print(df.DEVICETYPE.value_counts())

no of nan values :  0
Desktop    78423
Mobile     32065
Tablet      3425
Unknown       24
Name: DEVICETYPE, dtype: int64


In [ ]:
print("no of nan values : " ,df['PLATFORM_ID'].isna().sum())
print(df.PLATFORM_ID.value_counts())

no of nan values :  0
2     79151
7     20913
9      3891
5      3230
6      2978
10     1037
4       745
14      725
12      544
3       479
8       130
15       42
1        40
13       31
16        1
Name: PLATFORM_ID, dtype: int64


In [ ]:
print("no of nan values : " ,df['BIDREQUESTIP'].isna().sum())
print(df.BIDREQUESTIP.value_counts().head(60))

no of nan values :  0
71.105.120.171     1718
142.154.205.202    1143
68.132.98.211       866
108.175.114.100     727
108.175.114.101     707
173.66.114.27       675
162.17.102.153      629
12.218.110.242      604
65.216.253.25       581
150.148.14.137      517
99.160.6.197        505
129.176.151.15      456
23.125.217.54       426
65.112.8.18         419
24.102.87.62        373
69.92.49.4          350
38.240.111.51       346
47.189.55.180       327
71.87.24.123        303
67.203.36.94        259
198.184.147.58      257
128.146.52.108      253
67.203.36.81        244
71.66.42.242        237
128.146.189.103     231
209.126.104.10      218
50.26.129.135       211
98.110.76.22        209
66.249.66.75        207
24.45.121.35        202
66.249.66.73        200
209.126.104.7       199
207.38.89.236       183
207.38.87.145       182
209.126.104.18      176
209.126.127.71      169
185.228.195.71      169
35.129.213.194      169
209.126.124.58      168
216.24.45.43        168
66.97.189.250     

In [ ]:
print("no of nan values : " ,df['USERPLATFORMUID'].isna().sum())
print(df.USERPLATFORMUID.value_counts().head(20))

no of nan values :  4
3e2578c8-f794-41af-a38c-c5cfb3c0f014    1447
d76f7c0b-1a64-4d4b-b36d-86c832be8837    1406
d00f28d6-3a50-46cd-92f3-8601bd57ad0e     675
fcd3d327-0ad5-425e-8ea8-33ffe6a31543     629
fe53f32c-4646-4abe-afea-90b21d6b8531     617
82fcc028-f146-4efb-b95e-69fd62109bbc     520
73828378-8999-4670-a3bf-e5e78096301c     517
fa24a307-1001-4f5e-9d87-669e91737c07     505
f6890950-af86-4400-ab0b-e8498d0ba99e     418
a4706c21-3ce1-453d-b57c-1541c50d15f5     418
6b1794db-cc22-4f66-b185-ecd99146bb07     391
d9f89efd-0f02-4844-9719-098302c545df     365
c42db189-7d1b-4639-b9c6-9fa4d80be4bd     347
29903453-4c40-4f25-98a5-ddc554f71555     346
9fee45ce-0010-40e0-a272-6c1c4f86ea00     344
72cbb382-31dd-4e2a-9e9c-9d0f40a7d0a9     324
cd57e558-f19d-4cd8-afcc-d0fa18259173     300
63238044-b275-480b-a613-780d45d2f70a     294
b323edaf-33ac-44f0-a570-b1c342f806b9     271
456631b8-ca59-4e7a-96a0-53bcbf85eaef     259
Name: USERPLATFORMUID, dtype: int64


In [ ]:
print("no of nan values : " ,df['USERCITY'].isna().sum())
print(df.USERCITY.value_counts().head(60))

no of nan values :  6359
New York              4943
Brooklyn              3237
St Louis              2545
Los Angeles           2154
Houston               1747
Queens                1629
Arlington             1503
Chicago               1448
Philadelphia          1191
Columbus              1105
Miami                 1023
Phoenix                888
Fort Worth             884
Deer Park              883
San Francisco          881
Rochester              798
Dallas                 784
Memphis                756
Charlotte              695
Pittsburgh             669
Palm Beach Gardens     664
Seattle                650
Atlanta                630
Rockville              592
Durham                 577
Washington             565
Minneapolis            557
Springfield            556
Cleveland              555
Cambridge              538
Denver                 503
Tampa                  485
San Diego              478
Portland               463
Buena Park             431
Jacksonville           430
Orl

In [ ]:
print("no of nan values : " ,df['USERZIPCODE'].isna().sum())
print(df.USERZIPCODE.value_counts())

no of nan values :  4592
63169.0        2116
11226.0        1752
22202.0        1259
10001.0        1190
90060.0        1058
               ... 
56748.0           1
77021.0           1
74880.0           1
93630.0           1
760865488.0       1
Name: USERZIPCODE, Length: 11278, dtype: int64


In [ ]:
print("no of nan values : " ,df['USERAGENT'].isna().sum())


no of nan values :  2


In [ ]:
print("no of nan values : " ,df['PLATFORMTYPE'].isna().sum())
print(df.PLATFORMTYPE.value_counts())

no of nan values :  0
Online Medical Journal      110120
Online Learning Portal        2978
Medical News Platform          798
Electronic Health Record        40
Tele-medicine Platform           1
Name: PLATFORMTYPE, dtype: int64


In [ ]:
print("no of nan values : " ,df['CHANNELTYPE'].isna().sum())
print(df.CHANNELTYPE.value_counts())

no of nan values :  0
Website    113937
Name: CHANNELTYPE, dtype: int64


In [ ]:
print("no of nan values : " ,df['URL'].isna().sum())
print(df.URL.nunique())

no of nan values :  0
5231


In [ ]:
print("no of nan values : " ,df['KEYWORDS'].isna().sum())
print(df.KEYWORDS.nunique())

no of nan values :  0
2460


In [ ]:
print("no of nan values : " ,df['IS_HCP'].isna().sum())

no of nan values :  1


In [ ]:
print("no of nan values : " ,df['TAXONOMY'].isna().sum())
print(df.TAXONOMY.value_counts())

no of nan values :  81624
2084P0800X    7930
2084N0400X    6621
207Q00000X    3997
207R00000X    3190
208000000X     837
              ... 
163WC0400X       1
246QM0706X       1
1835N1003X       1
207RI0008X       1
207XX0801X       1
Name: TAXONOMY, Length: 207, dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113937 non-null  int64  
 1   DEVICETYPE       113937 non-null  object 
 2   PLATFORM_ID      113937 non-null  int64  
 3   BIDREQUESTIP     113937 non-null  object 
 4   USERPLATFORMUID  113933 non-null  object 
 5   USERCITY         107578 non-null  object 
 6   USERZIPCODE      109345 non-null  float64
 7   USERAGENT        113935 non-null  object 
 8   PLATFORMTYPE     113937 non-null  object 
 9   CHANNELTYPE      113937 non-null  object 
 10  URL              113937 non-null  object 
 11  KEYWORDS         113937 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(2), int64(2), object(10)
memory usage: 12.2+ MB


In [ ]:
df.dtypes

ID                   int64
DEVICETYPE          object
PLATFORM_ID          int64
BIDREQUESTIP        object
USERPLATFORMUID     object
USERCITY            object
USERZIPCODE        float64
USERAGENT           object
PLATFORMTYPE        object
CHANNELTYPE         object
URL                 object
KEYWORDS            object
TAXONOMY            object
IS_HCP             float64
dtype: object

In [ ]:
# plt.matshow(df.corr())
# plt.show()

In [ ]:
a=df[["TAXONOMY","IS_HCP"]]

In [ ]:
print(a.iloc[:50])

      TAXONOMY  IS_HCP
0          NaN     0.0
1          NaN     0.0
2          NaN     0.0
3   2084P0800X     1.0
4          NaN     0.0
5          NaN     0.0
6          NaN     0.0
7          NaN     0.0
8   2084P0804X     1.0
9          NaN     0.0
10         NaN     0.0
11         NaN     0.0
12         NaN     0.0
13         NaN     0.0
14         NaN     0.0
15         NaN     0.0
16         NaN     0.0
17         NaN     0.0
18         NaN     0.0
19         NaN     0.0
20         NaN     0.0
21  2084N0400X     1.0
22         NaN     0.0
23         NaN     0.0
24         NaN     0.0
25         NaN     0.0
26         NaN     0.0
27         NaN     0.0
28         NaN     0.0
29  2084V0102X     1.0
30         NaN     0.0
31         NaN     0.0
32         NaN     0.0
33         NaN     0.0
34         NaN     0.0
35  207RC0000X     1.0
36         NaN     0.0
37         NaN     0.0
38         NaN     0.0
39         NaN     0.0
40         NaN     0.0
41         NaN     0.0
42         

In [ ]:
df['TAXONOMY'].value_counts()

2084P0800X    7930
2084N0400X    6621
207Q00000X    3997
207R00000X    3190
208000000X     837
              ... 
163WC0400X       1
246QM0706X       1
1835N1003X       1
207RI0008X       1
207XX0801X       1
Name: TAXONOMY, Length: 207, dtype: int64

In [ ]:
def scale(x):
  sclr = StandardScaler()
  x = sclr.fit(x)
  return x

In [53]:
def ohe(df,col):
  ohe = OneHotEncoder()
  colm = ohe.fit_transform(df[col].values.reshape(-1,1)).toarray()
  feature_labels=ohe.categories_
  feature_labels=np.array(feature_labels).ravel()
  x=pd.DataFrame(colm,columns=feature_labels)
  df=pd.concat([df,x],axis=1)
  del df[col]
  return df

In [ ]:
def drp(df):
  df.drop(['ID','USERPLATFORMUID','USERCITY','USERAGENT','CHANNELTYPE','URL'],axis = 1,inplace = True)
  return df

In [ ]:
def vectorize(df):
  vectorizer_1 =TfidfVectorizer()
  feature_matrix=vectorizer_1.fit_transform(df["BIDREQUESTIP"])
  vectorized_bidrequestip=pd.DataFrame(feature_matrix.toarray())
  vectorizer_2 = TfidfVectorizer(tokenizer=lambda x: x.split('|'))
  tfidf_matrix = vectorizer_2.fit_transform(df['KEYWORDS'])
  vectorized_keywords = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer_2.get_feature_names_out())
  df = pd.concat([df,vectorized_bidrequestip,vectorized_keywords],axis = 1)
  df.drop(['BIDREQUESTIP','KEYWORDS'],axis = 1,inplace = True)
  return df,vectorizer_1,vectorizer_2

In [ ]:
df = vectorize(df)
df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ID,DEVICETYPE,PLATFORM_ID,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,1001,Desktop,2,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1002,Desktop,2,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003,Desktop,2,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1004,Desktop,3,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1005,Mobile,7,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Dropping certain fixed columns


In [ ]:
df = drp(df)
print('no of nan values:',df.isnull().sum())
df.head()

no of nan values: DEVICETYPE                 0
PLATFORM_ID                0
USERZIPCODE             4592
PLATFORMTYPE               0
TAXONOMY               81624
                       ...  
yaws                       0
yellow fever               0
zoonoses                   0
zostavax                   0
zoster sine herpete        0
Length: 1468, dtype: int64


,DEVICETYPE,PLATFORM_ID,USERZIPCODE,PLATFORMTYPE,TAXONOMY,IS_HCP,0,1,2,3,...,wheezing,whipple disease,whole genome sequencing,whooping cough,wound infection,yaws,yellow fever,zoonoses,zostavax,zoster sine herpete
0,Desktop,2,97206.0,Online Medical Journal,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Desktop,2,22202.0,Online Medical Journal,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Desktop,2,83654.0,Online Medical Journal,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Desktop,3,229114624.0,Online Medical Journal,2084P0800X,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Mobile,7,77008.0,Online Medical Journal,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Performing One Hot Encoding over certian columns

In [ ]:
df = ohe(df,"DEVICETYPE")
df = ohe(df,"PLATFORMTYPE")
df.head()

,PLATFORM_ID,USERZIPCODE,TAXONOMY,IS_HCP,0,1,2,3,4,5,...,zoster sine herpete,Desktop,Mobile,Tablet,Unknown,Electronic Health Record,Medical News Platform,Online Learning Portal,Online Medical Journal,Tele-medicine Platform
0,2,97206.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,22202.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,83654.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,229114624.0,2084P0800X,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,7,77008.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
df_hcp = df.drop(['TAXONOMY'],axis = 1)
df_taxonomy = df.drop(['IS_HCP'],axis = 1)

In [ ]:
print("na in df_hcp:",df_hcp.isnull().sum(),"na in df_taxonomy:",df_taxonomy.isnull().sum())

na in df_hcp: PLATFORM_ID                    0
USERZIPCODE                 4592
IS_HCP                         1
0                              0
1                              0
                            ... 
Electronic Health Record       0
Medical News Platform          0
Online Learning Portal         0
Online Medical Journal         0
Tele-medicine Platform         0
Length: 1474, dtype: int64 na in df_taxonomy: PLATFORM_ID                     0
USERZIPCODE                  4592
TAXONOMY                    81624
0                               0
1                               0
                            ...  
Electronic Health Record        0
Medical News Platform           0
Online Learning Portal          0
Online Medical Journal          0
Tele-medicine Platform          0
Length: 1474, dtype: int64


Taking care of null values

In [ ]:
df_hcp.dropna(axis = 0,how = 'any',inplace = True)
df_taxonomy.dropna(axis = 0,how = 'any',inplace = True)
print(print("na in df_hcp:",df_hcp.isnull().sum(),"na in df_taxonomy:",df_taxonomy.isnull().sum()))

na in df_hcp: PLATFORM_ID                 0
USERZIPCODE                 0
IS_HCP                      0
0                           0
1                           0
                           ..
Electronic Health Record    0
Medical News Platform       0
Online Learning Portal      0
Online Medical Journal      0
Tele-medicine Platform      0
Length: 1474, dtype: int64 na in df_taxonomy: PLATFORM_ID                 0
USERZIPCODE                 0
TAXONOMY                    0
0                           0
1                           0
                           ..
Electronic Health Record    0
Medical News Platform       0
Online Learning Portal      0
Online Medical Journal      0
Tele-medicine Platform      0
Length: 1474, dtype: int64
None


In [ ]:
print("df_hcp_shape: ",df_hcp.shape," df_taxonomy_shape:",df_taxonomy.shape)

df_hcp_shape:  (109344, 1474)  df_taxonomy_shape: (32313, 1474)


df_hcp should be used to predict hcp classification problem and df_taxonomy should be used to train taxonomy prediction/classification

In [ ]:
#Hyperparameters
num_layers = 10
input_shape = ()
num_classes = 207 #207 taxonomy are there
units = [256,256,512,512,1024,1024,2048,2048,1024,1024] #list of size num_layers
dropout = [0.2,0.2,0.3,0.3,0.4,0.4,0.5,0.5,0.4,0.4] #list of size num_layers
num_epochs = 10

In [ ]:
def nn():
  input = Input(shape = input_shape)
  for i in range(num_layers):
    x = Dense(units[i],activation = 'relu')(input)
    input = Dropout(dropout[i])(x)
  x = Dense(units = 512,activation = 'relu')(input)
  y = Dense(units = num_classes,activation = 'softmax')(x)
  model = Model(inputs = input,outputs = y)
  model.compile(optimizer = tf.keras.optimizers.Adamdelta(rho = 0.9),loss ='sparse_categorical_crossentropy',metrics = ['accuracy'])
  return model

In [ ]:
model = nn()
#model.fit(X,y,validation_data = (X_val,y_val),epochs = num_epochs)